In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import util

In [15]:
dir_data = Path('/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/data/old-data-version/raw-new/Figure_1_2')
dir_output = Path('/Users/josh/Desktop/example_output_folder')

lst_dict_inputdata = [
    {'session_id': 'WT63_11082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT69_12192021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT69_12192021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},
]

dir_output.mkdir(parents=True, exist_ok=True)

columnName_alignment_trial_start = 'photometryCenterInIndex'
columnName_alignment_trial_end = 'photometrySideOutIndex'

# Note: Alignment values of 0 for Matlab-indexed trial tables will be treated as "no-data" values
# and and -1 for Python-indexed trial tables. Matlab-indexed trial tables should only have values
# >= 0 in and >= -1 in Python.
trial_index_columns = [
    'photometryCenterInIndex',
    'photometryCenterOutIndex',
    'photometrySideInIndex',
    'photometrySideOutIndex',
]

trial_num_columns = [
    'leftRewardProb',
    'rightRewardProb',
]

trial_cat_columns = [
    'nTrial',
    'hasAllPhotometryData',
    'wasRewarded',
    'word',
]

signal_dense_columns = [
    'Ch1',
    'Ch2',
    'Ch5',
    'Ch6',
]

signal_sparse_columns = [
    # 'centerOcc',
    # 'centerIn',
    # 'centerOut',
    # 'rightOcc',
    # 'rightIn',
    # 'rightOut',
    # 'rightLick',
    # 'leftOcc',
    # 'leftIn',
    # 'leftOut',
    # 'leftLick',
    'SEnt'
]

In [22]:
for dict_inputdata in lst_dict_inputdata:
    df_signal = pd.read_csv(dict_inputdata['filepath_signal'])
    df_trial = pd.read_csv(dict_inputdata['filepath_trial'])
    df_trial['nTrial'] = np.arange(df_trial.shape[0])

    nrows = df_signal.shape[0]
    
    df_trial = df_trial[(df_trial[trial_index_columns] == 0).sum(axis=1) == 0]
    df_trial_indices = df_trial[trial_index_columns]
    
    matrix_signal_sparse = util.Matrix.from_pd_dense_num(df_signal[signal_sparse_columns])
    matrix_signal_dense = util.Matrix.from_pd_dense_num(df_signal[signal_dense_columns])
    matrix_trial_inx = util.Matrix.from_pd_sparse_num(nrows, indices=df_trial_indices - 1, values=None, sparse_value=0)
    matrix_trial_num = util.Matrix.from_pd_sparse_num(nrows, indices=df_trial_indices - 1, values=df_trial[trial_num_columns], sparse_value=np.nan)
    matrix_trial_cat = util.Matrix.from_pd_sparse_num(nrows, indices=df_trial_indices - 1, values=df_trial[trial_cat_columns], sparse_value=np.nan)
    
    matrix_all_preds = util.Matrix([matrix_signal_sparse, matrix_signal_dense, matrix_trial_num, matrix_trial_cat])

    # print(matrix_all_preds.vectors['centerOcc'])
    
    mtc = matrix_trial_cat.to_pd()['nTrial']
    display(mtc[(~mtc.isna()).sum(axis=1) > 0])
    # display(matrix_trial_cat.to_pd()['nTrial'])
    display(matrix_trial_inx.to_pd())

,photometryCenterInIndex,photometryCenterOutIndex,photometrySideInIndex,photometrySideOutIndex
1222,19.0,NaN,NaN,NaN
1225,NaN,19.0,NaN,NaN
1239,NaN,NaN,19.0,NaN
1265,NaN,NaN,NaN,19.0
1290,20.0,NaN,NaN,NaN
...,...,...,...,...
29070,NaN,NaN,NaN,344.0
29104,345.0,NaN,NaN,NaN
29107,NaN,345.0,NaN,NaN
29128,NaN,NaN,345.0,NaN


\
      photometryCenterInIndex photometryCenterOutIndex photometrySideInIndex   
0                         0.0                      0.0                   0.0   
1                         0.0                      0.0                   0.0   
2                         0.0                      0.0                   0.0   
3                         0.0                      0.0                   0.0   
4                         0.0                      0.0                   0.0   
...                       ...                      ...                   ...   
29845                     0.0                      0.0                   0.0   
29846                     0.0                      0.0                   0.0   
29847                     0.0                      0.0                   0.0   
29848                     0.0                      0.0                   0.0   
29849                     0.0                      0.0                   0.0   

                              
      photometrySideOutIndex  
0                        0.0  
1                        0.0  
2                        0.0  
3                        0.0  
4                        0.0  
...                      ...  
29845                    0.0  
29846                    0.0  
29847                    0.0  
29848                    0.0  
29849                    0.0  

[29850 rows x 4 columns]